In [3]:
import pandas as pd
import numpy as np

from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### Entitiy Matching

In [4]:
shipper_matching = pd.read_csv('../shipper_matching/Enigma_Enigma_6countries.csv')

In [5]:
shipper_matching.shape

(420944, 12)

In [12]:
shipper_matching[((shipper_matching['name_score']>0.9) & (shipper_matching['address_score']>0.6)) | (shipper_matching['address_score']>0.9)].shape

(37092, 12)

In [13]:
shipper_matching[((shipper_matching['name_score']>0.9) & (shipper_matching['address_score']>0.6))].shape

(33266, 12)

In [ ]:
shipper_matching  = shipper_matching[(shipper_matching['name_score']>0.9) | (shipper_matching['address_score']>0.6)]

In [ ]:
def shipper_frozenset(col1,col2):
    return frozenset([col1,col2])
def shipper_set(col1,col2):
    return {col1,col2}

In [ ]:
shipper_frozenset_vectorize = np.vectorize(shipper_frozenset)
shipper_matching['cl_shipper_frozenset'] = shipper_frozenset_vectorize(shipper_matching['cl_shipper_party_name_left'],shipper_matching['cl_shipper_party_name_right']) 
# elinamate left right mirror
shipper_matching.drop_duplicates(subset='cl_shipper_frozenset')

In [ ]:
shipper_matching['cl_shipper_frozenset'].shape

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(shipper_matching['cl_shipper_party_name_left'])
shipper_matching['cl_shipper_left_index'] = le.transform(shipper_matching['cl_shipper_party_name_left'].values)
shipper_matching['cl_shipper_riht_index'] = le.transform(shipper_matching['cl_shipper_party_name_right'].values)
#inverse_transform = le.inverse_transform(shipper_matching['cl_shipper_party_name_right'].values)

In [ ]:
shipper_matching.head()

In [1]:
if not {'a','b'}.isdisjoint({'b','d'}):
    a = {'a','b'}.union({'b','d'})
else:
    a = None
a

{'a', 'b', 'd'}

In [18]:
str(frozenset(a))[10:-1]

"{'b', 'd', 'a'}"

In [ ]:
def create_set_master(master_left,master_right):
    if not master_left.isdisjoint(master_right):
        new_master = master_left.union(master_right)
    else:
        new_master = master_left
    return new_master

def multiprocess_set(df,step):
    create_master_set_vectorize = np.vectorize(create_master_set)
    dfL_join_dfR = pd.DataFrame(np.roll(df,step,axis=0),columns=df.columns).join(df['cl_shipper_set'],lsuffix='_left',rsuffix='_right')
    dfL_join_dfR['master_set'] = create_master_set_vectorize(dfL_join_dfR['cl_shipper_party_name_left'].values,dfL_join_dfR['cl_shipper_party_name_right'].values)
    dfL_join_dfR = dfL_join_dfR[dfL_join_dfR['name_score']>=0.80]
    #result = result.loc[(result['score'] < 0.80) & (result['score']>=0.65)]
    #print('shift step {} done'.format(step))
    return dfL_join_dfR

In [ ]:
shipper_set_vectorize = np.vectorize(shipper_set)
create_set_master_vectorize = np.vectorize(create_set_master)
shipper_matching['cl_shipper_set'] = shipper_set_vectorize(shipper_matching['cl_shipper_party_name_left'].values,shipper_matching['cl_shipper_party_name_right'].values)
shipper_matching['cl_shipper_set_master'] = shipper_matching['cl_shipper_set'].copy()
shipper_matching_copy = shipper_matching.copy()

shift_steps = [i for i in range(len(shipper_matching))]
for step in shift_steps:
    #shipper_matching = pd.DataFrame(np.roll(shipper_matching,step,axis=0),columns=shipper_matching.columns).join(shipper_matching_copy['cl_shipper_set'],rsuffix='_right')
    shipper_matching['cl_shipper_set_right'] = np.roll(shipper_matching_copy['cl_shipper_set'],step)
    shipper_matching['cl_shipper_set_master'] = create_set_master_vectorize(shipper_matching['cl_shipper_set_master'].values,shipper_matching['cl_shipper_set_right'].values)
    #print('step {} done'.format(step))

In [ ]:
shipper_matching['cl_shipper_set_master'].iloc[3]

### Logistics Company

In [ ]:
data = read_data()

In [ ]:
data = process_data(data)

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data['shipper_party_name'].count()

In [ ]:
data['harmonized_number'].count()

In [ ]:
data['shipper_party_name'].str.contains(pat='logistic|logistics|forward|dhl|transport|shipping',case=False).sum()